## with out postprocess

In [ ]:
!ls ../input/

In [ ]:
!python ../input/gq-inf-scripts/tf_bert_large_inf.py

In [ ]:
!python ../input/gq-inf-scripts/pytorch_bert_inf.py

In [ ]:
!python ../input/gq-inf-scripts/pytorch_albert_inf.py

In [ ]:
!python ../input/gq-inf-scripts/tf_bert_base_inf.py

In [ ]:
# !python ../input/gq-inf-scripts/pytorch_base_inf.py

In [ ]:
!python ../input/gq-inf-scripts/pytorch_feat_inf.py

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import spearmanr
from scipy.optimize import minimize
from collections import defaultdict

kf = np.load("../input/split-dataset/dataset_indexs.npy", allow_pickle=True)

In [ ]:
train_df = pd.read_csv('../input/google-quest-challenge/train.csv')
sub_df = pd.read_csv('../input/google-quest-challenge/sample_submission.csv')

In [ ]:
# model_names = ["albert", "bert_base", "feat_multi", "feat_nn", "pytorch_base",
#                "tf_bert_base_epoch1", "tf_bert_base_epoch2", "tf_bert_base_epoch3"]
model_names = ["albert", "bert_base", "tf_bert_large", "feat_multi", "feat_nn",
               "tf_bert_base_epoch1", "tf_bert_base_epoch2", "tf_bert_base_epoch3"]

In [ ]:
def compute_spearmanr(trues, preds):
    rhos = []
    for col_true, col_pred in zip(trues.T, preds.T):
        rhos.append(spearmanr(col_true, col_pred + np.random.normal(0, 1e-7, col_pred.shape[0])).correlation)
    return np.mean(rhos)

# def compute_spearmanr(trues, preds, columns=None):
#     rhos = []
#     for i, (col_true, col_pred) in enumerate(zip(trues.T, preds.T)):
#         if i == 0:
#             # qa_id
#             continue
#         r = spearmanr(col_true, col_pred).correlation
#         if columns:
#             print("{:.3f}\t{}".format(r, columns[i]))
#         rhos.append(r)
#     return np.mean(rhos)

In [ ]:
target_columns = list(sub_df.columns)

results = []
for fold_i in range(1, 6):
    result_df = pd.DataFrame(index=model_names, columns=target_columns[-30:] + ["ave"])
    for model_name in model_names:
        for target in target_columns[-30:]:
            true = train_df[target_columns].iloc[kf[fold_i-1][1]].reset_index(drop=True)
            oof = pd.read_csv(model_name + f"/oof{fold_i}.csv")
            r = spearmanr(true[target].values, oof[target].values + np.random.normal(0, 1e-7, oof.shape[0])).correlation
            result_df.loc[model_name, target] = r
        result_df.loc[model_name, "ave"] = np.mean(result_df.loc[model_name, target_columns[-30:]].values)
    results.append(result_df)


In [ ]:
results[0]

In [ ]:
# optimize model weight
kfold = 5
model_number = len(model_names)

true_dfs = []
for i in range(5):
    oof = train_df.iloc[kf[i][1]]
    true_dfs.append(oof[target_columns])

model_df = defaultdict(lambda: [])
for model_name in model_names:
    print(model_name)

    scores = []
    pred_dfs = []

    for i in range(5):
        pred = pd.read_csv(f"{model_name}/oof{i+1}.csv")
        model_df[model_name].append(pred)
        true = true_dfs[i]
        rho = compute_spearmanr(true.values, pred.values)
        scores.append(rho)
        pred_dfs.append(pred.copy())
    print([f"fold{i+1}: {scores[i]:.4}" for i in range(5)])
    
scores = [{} for _ in range(kfold)]
all_coeffs = []

for col_num, col_name in enumerate(target_columns[-30:]):
    coefficients = np.zeros((kfold, model_number))
    print(col_name, "\n")
    for fold_num, (_, vld_idx) in enumerate(kf): 
        print("\n"+"-"*50+'\n[Fold {}/{}]'.format(fold_num + 1, kfold))
        # optimize function
        def function_spearmanr(x):
            true = true_dfs[fold_num].loc[:, col_name].values
            pred = np.zeros(len(true))
            for k, (model_name, df) in enumerate(model_df.items()):
                pred += x[k] * df[fold_num].loc[:, col_name].values
            return -spearmanr(true, pred).correlation

        # initial weights
        x0 = [1 for _ in range(model_number)]
        # optimize
        res = minimize(function_spearmanr, x0, method='nelder-mead', options={'xtol': 1e-4, 'disp': True})
        # normalize sum(weights) = 1.0
        coeffs = [max(val, 0) for val in res.x]
        coefficients[fold_num,:] = coeffs / np.sum(coeffs)
        print('\nfinal vector: {}'.format(coefficients[fold_num,:]))

        y_true = true_dfs[fold_num].loc[:, col_name].values
        y_pred = np.zeros(len(y_true))
        for k, (model_name, df) in enumerate(model_df.items()):
            y_pred += coefficients[fold_num, k] * df[fold_num].loc[:, col_name].values

        score = spearmanr(y_true, y_pred).correlation
        scores[fold_num][col_name] = score
        print(f"valid's spearmanr: {score:<7.4f} \n")
    all_coeffs.append(coefficients)


In [ ]:
np.mean(np.array([list(s.values()) for s in scores]))

In [ ]:
scores

In [ ]:
# ensamble
sub_df = pd.read_csv("../input/google-quest-challenge/sample_submission.csv")
submission = np.zeros((len(sub_df), 30)).T

# load each submission.csv
FOLD_NUM = 5
inf_sub = []
for fold_num in range(FOLD_NUM):
    l = []
    for model_num, model_name in enumerate(model_names):
        l.append(pd.read_csv(f"{model_name}/submission{fold_num+1}.csv"))
    inf_sub.append(l.copy())

In [ ]:
for col_num, col_name in enumerate(target_columns[-30:]):
    for fold_num in range(FOLD_NUM): 
        for model_num in range(len(model_names)):
            coef = all_coeffs[col_num][fold_num][model_num]
            submission[col_num] += coef / FOLD_NUM * inf_sub[fold_num][model_num][col_name].values

sub_df.iloc[:, -30:] = submission.T
sub_df

In [ ]:
print(np.max(sub_df.iloc[:, -30:].values), np.min(sub_df.iloc[:, -30:].values))

In [ ]:
def norm_sub(df):
    for col_name in df.columns[-30:]:
        tmp_df = df[col_name].values
        v_max = np.max(tmp_df) + 0.01
        v_min = np.min(tmp_df) - 0.01
        df[col_name] = df[col_name].apply(lambda x: (x - v_min) / (v_max - v_min))
        df[col_name] = df[col_name].values + np.random.normal(0, 1e-7, len(df))
    return df
sub_df = norm_sub(sub_df)

In [ ]:
print(np.max(sub_df.iloc[:, -30:].values), np.min(sub_df.iloc[:, -30:].values))

In [ ]:
sub_df

In [ ]:
# # check bug
# debug_sub_files = ["submission_230.csv", "submission_253.csv", "submission_288.csv", "submission_301.csv", 
#                    "submission_305.csv", "submission_335.csv", "submission_342.csv", "submission_350.csv"]
# for model_num, model_name in enumerate(model_names):
#     try:
#         for i in range(1, 6):
#             tmp = pd.read_csv(f"{model_name}/oof{i}.csv")
#             tmp = pd.read_csv(f"{model_name}/submission{i}.csv")
#     except:
#         debug_sub = pd.read_csv(f"../input/gq-debug/{debug_sub_files[model_num]}")
#         sub_df = pd.merge(sub_df["qa_id"], debug_sub, on="qa_id", how='outer')
#         sub_df = sub_df.fillna(0)
#         sub_df = norm_sub(sub_df)
#         break
# sub_df.to_csv("submission.csv", index=False)

In [ ]:
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

def compute_spearmanr(trues, preds, columns=None):
    rhos = []
    for i, (col_true, col_pred) in enumerate(zip(trues.T, preds.T)):
        if i == 0:
            # qa_id
            continue
        r = spearmanr(col_true, col_pred).correlation
        if columns:
            print("{:.3f}\t{}".format(r, columns[i-1]))
        rhos.append(r)
    return np.mean(rhos)

def compare_spearmanr(trues, preds, pp_preds, columns=None):
    rhos = []
    pp_rhos = []
    for i, (col_true, col_pred, col_pp_pred) in enumerate(zip(trues.T, preds.T, pp_preds.T)):
        if i == 0:
            # qa_id
            continue
        r = spearmanr(col_true, col_pred).correlation
        pp_r = spearmanr(col_true, col_pp_pred).correlation
        if columns:
            if r < pp_r:
                print("{:.3f} ->".format(r) + color.BOLD +  " {:.3f}".format(pp_r) + color.END + " \t{}".format(columns[i-1]))
            elif (r - pp_r) < 10**(-6):
                print("{:.3f} ->".format(r) + " {:.3f}".format(pp_r) +  " \t{}".format(columns[i-1]))
            else:
                print("{:.3f} ->".format(r) + color.RED +  " {:.3f}".format(pp_r) + color.END + " \t{}".format(columns[i-1]))

        rhos.append(r)
        pp_rhos.append(pp_r)
    whole_base = np.mean(rhos)
    whole_pp = np.mean(pp_rhos)
    print("Whole Score")
    if whole_base < whole_pp:
        print("{:.3f} ->".format(whole_base) + color.BOLD +  " {:.3f}".format(whole_pp) + color.END + " +{}".format(abs(whole_pp - whole_base)))
    elif (whole_base - whole_pp ) < 10**(-6):
        print("{:.3f} ->".format(whole_base) + " {:.3f}".format(whole_pp))
    else:
        print("{:.3f} ->".format(whole_base) + color.RED +  " {:.3f}".format(whole_pp) + color.END + " -{}".format(abs(whole_pp - whole_base)))
    return whole_base, whole_pp

def inf_numbert_of_annotators(v):
    s = set(v)
    d = 10

    for v in s:
        for u in s:
            if v != u:
                d = min(abs(v - u), d)
    for i in range(100):
        if d * i >= 1 - 10**(-3):
            return i

def zero_one_scale(v):
    max_x = np.max(v)
    min_x = np.min(v)
    return (v - min_x) / (max_x - min_x)


def post_process(train_df, train, pred_df):
    print(len(train), len(pred_df))
    df = pred_df.copy()
    # question_type_spelling not english or ell
    df.loc[(train["host"] != "english.stackexchange.com") & (train["host"] != "ell.stackexchange.com"), "question_type_spelling"] = 0

    # 0-1 scaling　
    for col in ["question_type_spelling", "question_type_entity", "question_type_definition", 
                "question_type_compare", "question_conversational", "question_interestingness_others",
                "question_type_instructions", "question_type_consequence", "question_interestingness_self",
                "question_has_commonly_accepted_answer", "answer_relevance", "question_not_really_a_question"]:
#     for col in target_columns[-30:]:
        n = inf_numbert_of_annotators(train_df[col].values)
        df[col] = zero_one_scale(df[col].values)
        for i in range(n+1):
            base = i / n
            low = base - 1 / (2 * n)
            up  = base + 1 / (2 * n)
            df.loc[(df[col] >= low) & (df[col] <= up), col] = base

        if len(set(df[col].values)) == 1:
            df[col] = pred_df[col].values

    return df

In [ ]:
train_df = pd.read_csv('../input/google-quest-challenge/train.csv')
test_df = pd.read_csv('../input/google-quest-challenge/test.csv')

# sub_df = post_process(train_df, test_df, sub_df)

In [ ]:
sub_df.to_csv("submission.csv", index=False)